# Baseline

##Что такое baseline?

**Бейзлайн (от анг. baseline - базовая линия)** - основная простая модель, которая может служить ориентиром для дальнейшей работы и оценки качества. На хакатонах такие модели обычно служат отправной точкой для написания своего кода.

Также бейзлайны позволяют понять, какой итоговый результат должна давать модель. Например, в нашем случае это файл для отправки на платформу.


##Как работать с бейзлайном?

1) Запустите бейзлайн, получите ответ и отправьте его на плфторму проведения, чтобы посмотреть на изначальные результаты работы, а также обозначить себя на лидерборде.




2) Внимательно познакомьтесь с данными, поищите дополнительную информацию в интернете на тему данных, разберитесь в признаках и теме.




3) Попробуйте улучшить предобработку данных: сделайте визуализации, ставьте и проверяйте гипотезы. Для проверки ваших гипотез отправляйте результат на платформу и следите за изменением вашего места на турнирной таблице.




4) Подумайте о том, какими инструментами лучше решить представленную задачу машинного обучения. В нашем случае это задача бинарной классификации.

##Код бейзлайна.

Сначала выполним импорт всех необходимых библиотек.

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

Здесь мы открываем файлы из папки data: данные для обучения модели и для предсказания.

In [36]:
model = pd.read_csv('/content/For_model.csv')
check = pd.read_csv('/content/For_predict.csv')

Проверяем, что всё загрузилось корректно и выводим первые 10 строк.

In [37]:
model.head(10)

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,6.901420,24.569658,0.862829,0.737416,G5V,20.449126,1
1,4.334196,2.198947,0.593309,1.153210,G2Ib,13.043324,0
2,7.873019,11.750465,0.616568,0.589166,F5V,17.736492,1
3,8.110014,2.143815,0.848508,1.322107,K2III,15.592469,0
4,8.025193,4.165622,0.899296,1.141646,K5III,16.379680,0
5,2.899130,11.110086,0.701719,1.095357,M0III,14.370388,0
6,7.529711,9.160176,1.131393,0.926519,G8IV,15.938899,1
7,6.083824,3.928857,0.835288,1.761544,M2.5IIIa,14.063334,0
8,3.000914,5.713252,0.757593,0.113611,A0V,14.350210,1
9,8.706528,1.816360,1.409039,0.250017,A0V,14.763104,1


In [38]:
check.head(10)

,Vmag,Plx,e_Plx,B-V,SpType,Amag
0,4.039519,13.596504,0.845101,-0.097823,F5V+...,15.774122
1,6.655719,4.010332,1.050017,1.386670,K5III,15.344976
2,6.405060,0.348855,0.961598,0.951503,B0IVpe,12.236103
3,9.113068,2.954461,1.251422,1.002819,K1III,15.560838
4,6.107686,3.144180,0.475795,0.493345,G8II-IIIvar,12.771981
5,7.337771,12.254347,0.651041,0.330493,F4V,17.315567
6,7.316560,0.662464,0.720464,-0.056743,B4V:ne,12.845317
7,7.379830,4.494505,0.594863,1.322639,K2III,15.528721
8,9.284617,7.392761,1.116969,0.629743,G3V,19.626463
9,8.646860,4.137044,1.239575,0.371263,F6V:,17.284021


Сразу заметим, что в разных файлах разное количество столбцов!

Нам важно проверить, нет ли в наших данных пропусков. Сначала посмотрим те данные, на которых мы будем обучать модель.

In [39]:
model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31641 entries, 0 to 31640
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Vmag         31641 non-null  float64
 1   Plx          31641 non-null  float64
 2   e_Plx        31641 non-null  float64
 3   B-V          31641 non-null  float64
 4   SpType       31641 non-null  object 
 5   Amag         31641 non-null  float64
 6   TargetClass  31641 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.7+ MB


Как мы видим, пропусков нет! Значит, заполнять их нам не нужно.

Однако, замечаем, что у нас в данных есть нечисловой признак! Это "SpType",  спектральный класс звезды согласно классификации Моргана-Кинана. Необходимо преобразовать этот признак в числовой.

In [40]:
le = LabelEncoder()
label = le.fit_transform(model['SpType'])
label

array([1256, 1159,  957, ...,  891, 1439, 1195])

In [41]:
model.drop("SpType", axis=1, inplace=True)
model["SpType"] = label

Здесь это сделано далеко не самым оптимальным способом. Для того, чтобы улучшить обработку этого признака, пожалуйста, познакомьтесь с физическим смыслом признака и данных, которые в нём содержатся.

Проверим результат.

In [62]:
model.head(10)

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,SpType
0,6.901420,24.569658,0.862829,0.737416,20.449126,1,1256
1,4.334196,2.198947,0.593309,1.153210,13.043324,0,1159
2,7.873019,11.750465,0.616568,0.589166,17.736492,1,957
3,8.110014,2.143815,0.848508,1.322107,15.592469,0,1572
4,8.025193,4.165622,0.899296,1.141646,16.379680,0,1669
5,2.899130,11.110086,0.701719,1.095357,14.370388,0,1701
6,7.529711,9.160176,1.131393,0.926519,15.938899,1,1371
7,6.083824,3.928857,0.835288,1.761544,14.063334,0,1723
8,3.000914,5.713252,0.757593,0.113611,14.350210,1,26
9,8.706528,1.816360,1.409039,0.250017,14.763104,1,26


Перед обучением важно проверять баланс данных: нет ли такого, что одному классу принадлежит более 70% строк. В наших данных дисбаланса нет.

In [43]:
model['TargetClass'].value_counts()

0    15842
1    15799
Name: TargetClass, dtype: int64

Делим наши данные на train и test. Не забываем, что нам нужно предсказать является ли звезда гигантом или карликом.

In [44]:
y = model['TargetClass']
X = model.drop(columns = ['TargetClass'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

Модель, выбранная в бейзлайне - случайный лес. Она не является оптимальной, но быстро обучается и позволяет получить результат, который выше случайного. Можете начать улучшать бейзлайлн с того, что попробовать изменить гиперпараметры представленного случайного леса.

In [45]:
%%time
rfc = RandomForestClassifier(random_state = 42, n_jobs = -1)
rfc.fit(X_train, y_train)

CPU times: user 6.84 s, sys: 30.5 ms, total: 6.87 s
Wall time: 3.65 s


RandomForestClassifier(n_jobs=-1, random_state=42)

Получаем результат. 

In [46]:
pred = rfc.predict(X_test)

Здесь мы оцениваем то, насколько точно работает наша модель. Здесь мы смотрим на ROC-AUC метрику.

In [47]:
roc_auc_score(y_test, pred)

0.9142484040754112

Далее мы обращаемся к данным, для которых нам необходимо сделать предсказание и применить модель.

In [48]:
check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7911 entries, 0 to 7910
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Vmag    7911 non-null   float64
 1   Plx     7911 non-null   float64
 2   e_Plx   7911 non-null   float64
 3   B-V     7911 non-null   float64
 4   SpType  7911 non-null   object 
 5   Amag    7911 non-null   float64
dtypes: float64(5), object(1)
memory usage: 371.0+ KB


Применяем разработанную модель к нашим данным, но до этого необходимо преобразоват признак "SpType".

In [49]:
label = le.fit_transform(check['SpType'])
label

array([ 620, 1027,  231, ...,  709,  560,  931])

In [50]:
check.drop("SpType", axis=1, inplace=True)
check["SpType"] = label

In [51]:
pred = rfc.predict(check)

Сохарняем результат в таком формате, чтобы его затем можно было бы вывести как CSV файл и выгружаем итоговый результат.

In [52]:
pred = pd.Series(pred)

In [53]:
pred.head(10)

0    1
1    0
2    0
3    0
4    0
5    1
6    0
7    0
8    1
9    1
dtype: int64

In [54]:
pred.to_csv('Result.csv')
pred_file = pd.read_csv('/content/Result.csv')
pred_file.head(10)

,Unnamed: 0,0
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0
5,5,1
6,6,0
7,7,0
8,8,1
9,9,1


In [55]:
pred_file.rename(columns={'0': 'TargetClass','Unnamed: 0':'Number'}, inplace=True)

In [56]:
pred_file.head(10)

,Number,TargetClass
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0
5,5,1
6,6,0
7,7,0
8,8,1
9,9,1


Удалим ненужные столбцы.

In [57]:
check=check.join(pred_file)

In [58]:
check.head()

,Vmag,Plx,e_Plx,B-V,Amag,SpType,Number,TargetClass
0,4.039519,13.596504,0.845101,-0.097823,15.774122,620,0,1
1,6.655719,4.010332,1.050017,1.386670,15.344976,1027,1,0
2,6.405060,0.348855,0.961598,0.951503,12.236103,231,2,0
3,9.113068,2.954461,1.251422,1.002819,15.560838,931,3,0
4,6.107686,3.144180,0.475795,0.493345,12.771981,839,4,0


In [59]:
pred_final = check.drop(['Number'], axis='columns')

Выполним проверку.

In [60]:
pred_final.head(10)

,Vmag,Plx,e_Plx,B-V,Amag,SpType,TargetClass
0,4.039519,13.596504,0.845101,-0.097823,15.774122,620,1
1,6.655719,4.010332,1.050017,1.386670,15.344976,1027,0
2,6.405060,0.348855,0.961598,0.951503,12.236103,231,0
3,9.113068,2.954461,1.251422,1.002819,15.560838,931,0
4,6.107686,3.144180,0.475795,0.493345,12.771981,839,0
5,7.337771,12.254347,0.651041,0.330493,17.315567,595,1
6,7.316560,0.662464,0.720464,-0.056743,12.845317,351,0
7,7.379830,4.494505,0.594863,1.322639,15.528721,969,0
8,9.284617,7.392761,1.116969,0.629743,19.626463,757,1
9,8.646860,4.137044,1.239575,0.371263,17.284021,643,1


И сохраним результат в итоговый файл.

In [61]:
pred_final.to_csv('Result_final.csv',index=False)

Желаем вам успехов в НТО!